In [3]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from efficientnet_pytorch import EfficientNet

# ==============================
# SEED 설정 (재현성 확보)
# ==============================
seed = 2021
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ==============================
# Early Stopping 클래스
# ==============================
class EarlyStopping:
    def __init__(self, patience=10, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# ==============================
# SE Block
# ==============================
class SEBlock(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(SEBlock, self).__init__()
        self.global_avgpool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(in_channels, in_channels // reduction),
            nn.ReLU(),
            nn.Linear(in_channels // reduction, in_channels),
            nn.Sigmoid()
        )

    def forward(self, x):
        batch, channels, _, _ = x.size()
        out = self.global_avgpool(x).view(batch, channels)
        out = self.fc(out).view(batch, channels, 1, 1)
        return x * out

# ==============================
# ASPP Block
# ==============================
class ASPP(nn.Module):
    def __init__(self, in_channels):
        super(ASPP, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, in_channels, kernel_size=1)
        self.conv3_1 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1, dilation=1)
        self.conv3_6 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=6, dilation=6)
        self.conv3_12 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=12, dilation=12)
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))

        self.conv1x1_out = nn.Conv2d(in_channels * 4, in_channels, kernel_size=1)

    def forward(self, x):
        conv1 = self.conv1(x)
        conv3_1 = self.conv3_1(x)
        conv3_6 = self.conv3_6(x)
        conv3_12 = self.conv3_12(x)

        out = torch.cat([conv1, conv3_1, conv3_6, conv3_12], dim=1)
        out = self.conv1x1_out(out)
        return out

# ==============================
# Custom EfficientNet 모델
# ==============================
class CustomEfficientNet(nn.Module):
    def __init__(self, model_name="efficientnet-b0", num_classes=6, freeze_backbone=False):
        super(CustomEfficientNet, self).__init__()
        self.backbone = EfficientNet.from_pretrained(model_name)
        if freeze_backbone:
            for param in self.backbone.parameters():
                param.requires_grad = False

        in_features = self.backbone._fc.in_features
        self.backbone._fc = nn.Identity()

        self.aspp = ASPP(in_channels=in_features)
        self.se = SEBlock(in_channels=in_features)

        self.classifier = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.backbone.extract_features(x)
        x = self.aspp(x)
        x = self.se(x)
        x = nn.AdaptiveAvgPool2d((1, 1))(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# ==============================
# 데이터 로딩 및 설정
# ==============================
model_name = 'efficientnet-b0'
num_classes = 6
batch_size = 32
epochs = 50
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

image_size = EfficientNet.get_image_size(model_name)

transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset_path = 'C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel'
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# ==============================
# 모델, 손실 함수, 최적화기 설정
# ==============================
model = CustomEfficientNet(model_name=model_name, num_classes=num_classes, freeze_backbone=False).to(device)
criterion = nn.CrossEntropyLoss()  # Label Smoothing 제거
optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # AdamW 제거

# ==============================
# 학습 및 평가 함수
# ==============================
def train(model, loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
        _, pred = outputs.max(1)
        correct += (pred == labels).sum().item()
        total += labels.size(0)
    return running_loss / total, correct / total

def evaluate(model, loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
            _, pred = outputs.max(1)
            correct += (pred == labels).sum().item()
            total += labels.size(0)
    return running_loss / total, correct / total

# ==============================
# 학습 루프
# ==============================
best_test_acc = 0.0
best_epoch = 0
early_stopping = EarlyStopping(patience=10)

for epoch in range(epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    test_loss, test_acc = evaluate(model, test_loader, criterion)

    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_epoch = epoch + 1

    early_stopping(test_loss)
    if early_stopping.early_stop:
        print("Early stopping triggered")
        break

    print(f"[Epoch {epoch+1}/{epochs}] Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"  Test Loss: {test_loss:.4f}  | Test Acc: {test_acc:.4f}")

print(f"\nTraining complete! Best Test Accuracy: {best_test_acc:.4f} at Epoch {best_epoch}")


Loaded pretrained weights for efficientnet-b0
[Epoch 1/50] Train Loss: 1.2450 | Train Acc: 0.5523
  Test Loss: 1.9485  | Test Acc: 0.5932
[Epoch 2/50] Train Loss: 0.9328 | Train Acc: 0.7387
  Test Loss: 1.2187  | Test Acc: 0.7175
[Epoch 3/50] Train Loss: 0.7439 | Train Acc: 0.8178
  Test Loss: 10.1194  | Test Acc: 0.5085
[Epoch 4/50] Train Loss: 0.5320 | Train Acc: 0.8672
  Test Loss: 4.4004  | Test Acc: 0.7175
[Epoch 5/50] Train Loss: 0.5963 | Train Acc: 0.8333
  Test Loss: 2.0350  | Test Acc: 0.6610
[Epoch 6/50] Train Loss: 0.7247 | Train Acc: 0.8291
  Test Loss: 1.6229  | Test Acc: 0.7571
[Epoch 7/50] Train Loss: 0.4402 | Train Acc: 0.8983
  Test Loss: 1.3992  | Test Acc: 0.8192
[Epoch 8/50] Train Loss: 0.3270 | Train Acc: 0.9110
  Test Loss: 2.3525  | Test Acc: 0.7514
[Epoch 9/50] Train Loss: 0.4782 | Train Acc: 0.8517
  Test Loss: 1.4903  | Test Acc: 0.7458
[Epoch 10/50] Train Loss: 0.3515 | Train Acc: 0.9195
  Test Loss: 2.7347  | Test Acc: 0.7119
[Epoch 11/50] Train Loss: 0.3483

In [6]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from efficientnet_pytorch import EfficientNet

# ==============================
# SEED 설정 (재현성 확보)
# ==============================
seed = 2021
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ==============================
# Early Stopping 클래스
# ==============================
class EarlyStopping:
    def __init__(self, patience=10, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# ==============================
# SE Block
# ==============================
class SEBlock(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(SEBlock, self).__init__()
        self.global_avgpool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(in_channels, in_channels // reduction),
            nn.ReLU(),
            nn.Linear(in_channels // reduction, in_channels),
            nn.Sigmoid()
        )

    def forward(self, x):
        batch, channels, _, _ = x.size()
        out = self.global_avgpool(x).view(batch, channels)
        out = self.fc(out).view(batch, channels, 1, 1)
        return x * out

# ==============================
# ASPP Block
# ==============================
class ASPP(nn.Module):
    def __init__(self, in_channels):
        super(ASPP, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, in_channels, kernel_size=1)
        self.conv3_1 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1, dilation=1)
        self.conv3_6 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=6, dilation=6)
        self.conv3_12 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=12, dilation=12)
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))

        self.conv1x1_out = nn.Conv2d(in_channels * 4, in_channels, kernel_size=1)

    def forward(self, x):
        conv1 = self.conv1(x)
        conv3_1 = self.conv3_1(x)
        conv3_6 = self.conv3_6(x)
        conv3_12 = self.conv3_12(x)

        out = torch.cat([conv1, conv3_1, conv3_6, conv3_12], dim=1)
        out = self.conv1x1_out(out)
        return out

# ==============================
# Custom EfficientNet 모델
# ==============================
class CustomEfficientNet(nn.Module):
    def __init__(self, model_name="efficientnet-b0", num_classes=6, freeze_backbone=False):
        super(CustomEfficientNet, self).__init__()
        self.backbone = EfficientNet.from_pretrained(model_name)
        if freeze_backbone:
            for param in self.backbone.parameters():
                param.requires_grad = False

        in_features = self.backbone._fc.in_features
        self.backbone._fc = nn.Identity()

        self.aspp = ASPP(in_channels=in_features)
        self.se = SEBlock(in_channels=in_features)

        self.classifier = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.backbone.extract_features(x)
        x = self.aspp(x)
        x = self.se(x)
        x = nn.AdaptiveAvgPool2d((1, 1))(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# ==============================
# 데이터 로딩 및 설정
# ==============================
model_name = 'efficientnet-b0'
num_classes = 6
batch_size = 32
epochs = 50
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

image_size = EfficientNet.get_image_size(model_name)

transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset_path = 'C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel'
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# ==============================
# 모델, 손실 함수, 최적화기 설정
# ==============================
model = CustomEfficientNet(model_name=model_name, num_classes=num_classes, freeze_backbone=False).to(device)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)  # Label Smoothing 적용
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

# ==============================
# 학습 루프
# ==============================
best_test_acc = 0.0
best_epoch = 0
early_stopping = EarlyStopping(patience=10)
# 학습 함수
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 통계
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# 평가 함수
def test(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward
            outputs = model(images)
            loss = criterion(outputs, labels)

            # 통계
            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# 학습 및 검증 루프
best_test_acc = 0.0  # 최고 Test Accuracy 저장
best_epoch = 0       # 최고 Test Accuracy를 달성한 Epoch
early_stopping = EarlyStopping(patience=10)  # Early Stopping 설정

for epoch in range(epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_loss, test_acc = test(model, test_loader, criterion, device)

    # 최고 Test Accuracy 갱신
    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_epoch = epoch + 1

    # Early Stopping 체크
    early_stopping(test_loss)
    if early_stopping.early_stop:
        print("Early stopping triggered")
        break

    # Epoch 결과 출력
    print(f"Epoch {epoch+1}/{epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

# 최고 Test Accuracy 출력
print(f"\nTraining complete! Best Test Accuracy: {best_test_acc:.4f} at Epoch {best_epoch}")


Loaded pretrained weights for efficientnet-b0
Epoch 1/50
Train Loss: 1.3637, Train Accuracy: 0.5706
Test Loss: 1.6704, Test Accuracy: 0.5989
Epoch 2/50
Train Loss: 1.1600, Train Accuracy: 0.7387
Test Loss: 1.9818, Test Accuracy: 0.5932
Epoch 3/50
Train Loss: 0.8410, Train Accuracy: 0.8503
Test Loss: 1.0492, Test Accuracy: 0.8136
Epoch 4/50
Train Loss: 0.7371, Train Accuracy: 0.8927
Test Loss: 1.1587, Test Accuracy: 0.8192
Epoch 5/50
Train Loss: 0.6558, Train Accuracy: 0.9393
Test Loss: 0.9720, Test Accuracy: 0.8136
Epoch 6/50
Train Loss: 0.6509, Train Accuracy: 0.9421
Test Loss: 1.0920, Test Accuracy: 0.7966
Epoch 7/50
Train Loss: 0.6373, Train Accuracy: 0.9407
Test Loss: 1.1342, Test Accuracy: 0.8079
Epoch 8/50
Train Loss: 0.5960, Train Accuracy: 0.9590
Test Loss: 1.2206, Test Accuracy: 0.7910
Epoch 9/50
Train Loss: 0.6902, Train Accuracy: 0.9153
Test Loss: 1.0154, Test Accuracy: 0.8531
Epoch 10/50
Train Loss: 0.7007, Train Accuracy: 0.9181
Test Loss: 1.0269, Test Accuracy: 0.7627
Epo

EfficientNet을 백본으로 사용 (전이 학습)
ASPP & SE Block 추가 (다양한 스케일 & 채널 중요도 강화)
AdamW 최적화 (Adam + Weight Decay)
Label Smoothing 적용 (모델 과적합 방지)
Early Stopping 활성화 (과적합 방지 및 학습 시간 단축